In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# preliminary information .

* I edited the solution I made with the LightGBM algorithm before, by developing it with the xgboost algorithm. . 
* I tried to keep the work plain and simple.
* If you have any questions don't hesitate to ask .
* iqr was applied for outlier detection . visualization was also used .
* Since the number of outliers was less than 0.01 (21), it was deleted instead of filled.

# first look at the data.

In [ ]:
sample_submission=pd.read_csv("/kaggle/input/playground-series-s3e6/sample_submission.csv")
train=pd.read_csv("/kaggle/input/playground-series-s3e6/train.csv")
test=pd.read_csv("/kaggle/input/playground-series-s3e6/test.csv")

In [ ]:
out_dataset=pd.read_csv("/kaggle/input/paris-housing-price-prediction/ParisHousing.csv")

In [ ]:
train.head()

In [ ]:
train.drop(['id'], axis=1, inplace=True) # I'm deleting unnecessary columns for guesswork.

In [ ]:
train =pd.concat([train , out_dataset], sort=False)

In [ ]:
train.info() 

In [ ]:
train.iloc[:,1:18].corr()
#sns.heatmap(train.iloc[:,1:18], annot=True)

In [ ]:
def iqr(data,var):# outliers detecion .
    q1 = np.quantile(data[var], 0.25)
    q3 = np.quantile(data[var], 0.75)
    diff = q3 - q1
    lower_t = q1 - (1.5 * diff)
    upper_t = q3 + (1.5 * diff)
    return data[(data[var] < lower_t) | (data[var] > upper_t)]

# squareMeters

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize =(10, 7))
 
# Creating plot
plt.boxplot(train.squareMeters)
 
# show plot
plt.show()

In [ ]:
#aykırı değerler .
iqr(train,"squareMeters")

In [ ]:
train.squareMeters.sort_values(ascending=False)
# değişmiş hali.

In [ ]:
dict = {6071330 :99985 ,146181:99985 } # uçuk değerleri maximum değere eşitledim .
train=train.replace({"squareMeters": dict})

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure(figsize =(10, 7))
 
# Creating plot
plt.boxplot(train.squareMeters)
 
# show plot
plt.show()

# numberOfRooms

In [ ]:
#aykırı değerler .
iqr(train,"numberOfRooms")

In [ ]:
train.numberOfRooms.describe()

In [ ]:
fig = plt.figure(figsize =(10, 7))
 
# Creating plot
plt.boxplot(train.numberOfRooms)
 
# show plot
plt.show()

# hasYard

In [ ]:
#aykırı değerler .
iqr(train,"hasYard")

In [ ]:
train.hasYard.value_counts()

# hasPool

In [ ]:
#aykırı değerler .
iqr(train,"hasPool")

In [ ]:
train.hasPool.value_counts()

# floors

In [ ]:
#aykırı değerler .
iqr(train,"floors")

In [ ]:
fig = plt.figure(figsize =(10, 7))
 
# Creating plot
plt.boxplot(train.floors)
 
# show plot
plt.show()

In [ ]:
train.floors.describe()

In [ ]:
train.floors.sort_values(ascending=False)
# değişmiş hali.

In [ ]:
dict = {6000 : 100} # uçuk değerleri maximum değere eşitledim .
train=train.replace({"floors": dict})

In [ ]:
fig = plt.figure(figsize =(10, 7))
 
# Creating plot
plt.boxplot(train.floors)
 
# show plot
plt.show()

# cityCode

> şehir kodu üzerine oynama yapılmayacak .

In [ ]:
'''
Buradaki değerler sayısal bir büyüklük olmadığı için kolon silinecek .
'''

train.drop(['cityCode'], axis=1, inplace=True)

# cityPartRange

In [ ]:
#aykırı değerler .
iqr(train,"cityPartRange")

In [ ]:
fig = plt.figure(figsize =(10, 7))
 
# Creating plot
plt.boxplot(train.cityPartRange)
 
# show plot
plt.show()

# numPrevOwners

In [ ]:
fig = plt.figure(figsize =(10, 7))
 
# Creating plot
plt.boxplot(train.numPrevOwners)
 
# show plot
plt.show()

# made

In [ ]:
#aykırı değerler .
iqr(train,"made")

In [ ]:
train.made.describe()

In [ ]:
train.made.sort_values(ascending=False)[:10]

In [ ]:
dict = { 10000 : 2021 } # uçuk değerleri maximum değere eşitledim .
train=train.replace({"made": dict})

In [ ]:
train.made.sort_values(ascending=False)

In [ ]:
fig = plt.figure(figsize =(10, 7))
 
# Creating plot
plt.boxplot(train.made)
 
# show plot
plt.show()

# basement

In [ ]:
#aykırı değerler .
iqr(train,"basement")

In [ ]:
fig = plt.figure(figsize =(10, 7))
 
# Creating plot
plt.boxplot(train.basement)
 
# show plot
plt.show()

In [ ]:
train.basement.sort_values(ascending=False)[:10]

In [ ]:
Q1 = train.basement.quantile(0.25)

Q3 = train.basement.quantile(0.75)

IQR = Q3 - Q1

Lower_basement = Q1 - (1.5 * IQR)

Upper_basement = Q3 + (1.5 * IQR)

print(Upper_basement)

In [ ]:
dict = { 91992 : 10000 , 91978 : 10000 , 
        84333 : 10000  , 81851 :10000 } # uçuk değerleri maximum değere eşitledim .
train=train.replace({"basement": dict})

In [ ]:
fig = plt.figure(figsize =(10, 7))
 
# Creating plot
plt.boxplot(train.basement)
 
# show plot
plt.show()

# attic

In [ ]:
#aykırı değerler .
iqr(train,"attic")

In [ ]:
fig = plt.figure(figsize =(10, 7))
 
# Creating plot
plt.boxplot(train.attic)
 
# show plot
plt.show()

In [ ]:
Q1 = train.attic.quantile(0.25)

Q3 = train.attic.quantile(0.75)

IQR = Q3 - Q1

Lower_attic = Q1 - (1.5 * IQR)

Upper_attic = Q3 + (1.5 * IQR)

print(Upper_attic)

In [ ]:
train.attic.sort_values(ascending=False)[:10]

In [ ]:
dict = { 96381 : 10000, 
         71965 : 10000 , 
         71024 : 10000, 
         71001 : 10000 ,
         30000 : 10000
       } # uçuk değerleri maximum değere eşitledim .
train=train.replace({"attic": dict})

In [ ]:
fig = plt.figure(figsize =(10, 7))
 
# Creating plot
plt.boxplot(train.attic)
 
# show plot
plt.show()

# garage

In [ ]:
#aykırı değerler .
iqr(train,"garage")

In [ ]:
train.garage.describe()

In [ ]:
Q1 = train.garage.quantile(0.25)

Q3 = train.garage.quantile(0.75)

IQR = Q3 - Q1

Lower_garage = Q1 - (1.5 * IQR)

Upper_garage = Q3 + (1.5 * IQR)

print(Upper_garage)

In [ ]:
train.garage.sort_values(ascending=False)[:10]

In [ ]:
dict = { 2048 : 1000, 
         9017 : 1000
       } # uçuk değerleri maximum değere eşitledim .
train=train.replace({"garage": dict})

In [ ]:
train.garage.sort_values(ascending=False)[:10]

In [ ]:
fig = plt.figure(figsize =(10, 7))
 
# Creating plot
plt.boxplot(train.garage)
 
# show plot
plt.show()

# hasGuestRoom

In [ ]:
#aykırı değerler .
iqr(train,"hasGuestRoom")

# toplu analiz

In [ ]:
train.dropna(inplace=True)

In [ ]:
train.info()

In [ ]:
# train.drop(['id'], axis=1, inplace=True)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
X=train.iloc[:,:15]
y=train.price 

In [ ]:
# Splitting
train_X, test_X, train_y, test_y = train_test_split(X, y,
                      test_size = 0.25, random_state = 123)
  

In [ ]:
from xgboost import XGBRegressor
RegModel=XGBRegressor(max_depth=3, learning_rate=0.25 , n_estimators=500, objective='reg:linear', booster='gbtree')
 
#Printing all the parameters of XGBoost
print(RegModel)
 
#Creating the model on Training Data
XGB=RegModel.fit(train_X,train_y)
prediction=XGB.predict(test_X)

In [ ]:
mse = mean_squared_error(test_y, prediction)
print("MSE: ", mse)
print("RMSE: ", mse*(1/2.0))

In [ ]:
x=2000 
y=(x+300)
x_ax = range(len(test_y))
f = plt.figure()
f.set_figwidth(20)
f.set_figheight(10)
plt.scatter(x_ax[x:y], test_y[x:y], s=5, color="blue", label="original")
plt.plot(x_ax[x:y], prediction[x:y], lw=0.8, color="red", label="predicted")
plt.xlabel("example prediciton")
plt.legend()
plt.show()

# test

In [ ]:
test.drop(['id','cityCode'], axis=1, inplace=True)

In [ ]:
test_pred = XGB.predict(test)

In [ ]:
test_pred 

In [ ]:
sample_submission.price=test_pred

In [ ]:
sample_submission.to_csv('submission1.csv', index = False)
pd.read_csv('submission1.csv')